# Incidentes con armas

## Análisis descriptivo

In [ ]:
# 0. load data in DataFrames
import pandas as pd
gun_violence_df = pd.read_csv('../data/raw/gun-violence-data_01-2013_03-2018.csv')

Pequeña muestra de los datos

In [ ]:
gun_violence_df.head()

Forma de los datos

In [ ]:
gun_violence_df.shape

Variables y sus tipos

In [ ]:
gun_violence_df.dtypes

Resumen de información de las variables

In [ ]:
gun_violence_df.info()

Descripción estadística de los datos

In [ ]:
gun_violence_df.describe()

Finalmente, hacemos uso de la herramienta Pandas Profiling para dar un resumen de nuestro dataset.

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(gun_violence_df, title='Gun Violence Incidents', html={'style':{'full_width':True}})
profile.to_notebook_iframe()

In [ ]:
import os
os.makedirs('../reports/descriptive_analysis', exist_ok=True)
profile.to_file(output_file='../reports/descriptive_analysis/gun_violence_incidents_precleaning.html')

## Limpieza

### Tratamiento de los valores perdidos

Primero, veremos en qué columnas existen valores nulos y cuántos de ellos hay.

In [ ]:
gun_violence_df.isnull().sum()

Por tanto, las columnas con valores nulos son las siguientes:

In [ ]:
null_columns = gun_violence_df.columns[gun_violence_df.isnull().any()]
null_columns

Y sus tipos de datos son:

In [ ]:
gun_violence_df[null_columns].dtypes

Ahora, generaremos la proporción de valores nulos en cada columna para una visión a más alto nivel.

In [ ]:
null_proportions_df = pd.DataFrame(gun_violence_df[null_columns].isnull().sum() / gun_violence_df.shape[0], columns=['proportion']).sort_values(by='proportion', ascending=False)
null_proportions_df

Una estrategia típica a seguir para la imputación de valores nulos es usar la media de la variable en caso de ser numérica, y la moda en caso de ser categórica.

Las variables `participant_relationship`, `location_description` y `participant_name` poseen más de un 50% de nulos. Por tanto, no tiene sentido la imputación de la moda (por ser categóricas), ya que esta no representa a la población, pues hay más valores nulos que completos. Por tanto, introduciremos un valor `Unknown`.

In [ ]:
# Impute value 'Unknown' for null values in columns 'participant_relationship', 'location_description' and 'participant_name'
gun_violence_df['participant_relationship'].fillna('Unknown', inplace=True)
gun_violence_df['location_description'].fillna('Unknown', inplace=True)
gun_violence_df['participant_name'].fillna('Unknown', inplace=True)

Las variables `gun_stolen`, `gun_type` y `n_guns_involved` están directamente relacionadas. Según el número de armas involucradas, el tipo y si han sido robadas o no para cada una se indican en las otras dos columnas. Por ello, cuando el número de armas involucradas es nulo, las otras dos también. Por tanto, debemos seguir la misma estrategia para las tres columnas.

Sin embargo, `n_guns_involved` es una variable numérica, mientras que `gun_stolen` y `gun_type` son categóricas. Por ello, tampoco tiene sentido utilizar o bien la media o bien la moda para todas.

Finalmente, hemos valorado dos opciones:

- Introducir un valor `Unknown`.
- Utilizar una estrategia particular para el dominio.

Hemos decidido crear una estrategia propia para nuestro problema.

`gun_stolen` y `gun_type` están formateados de la siguiente manera:

    "0::<valor>||1::<valor>||...||<nº de armas involucradas - 1>::<valor>"

Además, en algunos registros existe una errata en la que se usa ':' en lugar de '::', o '|' en lugar de '||'.

Por tanto, nuestra estrategia consiste en:

1. Extraer estos valores correspondientes a cada arma involucrada del conjunto de datos.
2. Reemplazar los valores nulos de `n_guns_involved` por su media redondeada a un entero.
3. Reemplazar los valores nulos de `gun_stolen` y `gun_type` por una cadena en el formato establecido compuesta por _n_ armas, siendo _n_ la media redondeada que se ha imputado en los valores nulos de `n_guns_involved`, y cada una de ellas con el valor más frecuente para las armas involucradas.

In [ ]:
from statistics import mode

# Function to extract individual values from the given format
def extract_values(column):
    values = []
    for row in column:
        if pd.isnull(row):
            continue
        # Remove '||' and '|' separators
        no_bars = list(filter(None, row.split('|')))
        for item in no_bars:
            if ':' in item:
                # Remove '::' and ':' separators
                no_separator = list(filter(None, item.split(':')))
                values.append(no_separator[1])
            # incident_characteristics column has a different format
            else:
                values.append(item)
        
    return values

# Extract modes for gun_stolen and gun_type columns
gun_stolen_mode = mode(extract_values(gun_violence_df['gun_stolen']))
gun_type_mode = mode(extract_values(gun_violence_df['gun_type']))

print("Moda de 'gun_stolen':", gun_stolen_mode)
print("Moda de 'gun_type':", gun_type_mode)

In [ ]:
n_guns_involved_mean = gun_violence_df['n_guns_involved'].mean()
n_guns_involved_mean = round(n_guns_involved_mean, 0)
print("Media redondeada de 'n_guns_involved':", n_guns_involved_mean)

Ahora, aplicaremos la estrategia mencionada.

In [ ]:
def format_values(values):
    formatted_values = ""
    n_values = len(values)
    for i in range(n_values):
        if i == n_values - 1:
            formatted_values += str(i) + "::" + str(values[i])
        else:
            formatted_values += str(i) + "::" + str(values[i]) + "||"
    return formatted_values

# Impute values for null values in columns 'gun_stolen', 'gun_type' and 'n_guns_involved'
gun_violence_df['n_guns_involved'].fillna(n_guns_involved_mean, inplace=True)
gun_violence_df['gun_stolen'].fillna(format_values([gun_stolen_mode]), inplace=True)
gun_violence_df['gun_type'].fillna(format_values([gun_type_mode]), inplace=True)

Observando el resto de columnas con valores nulos, vemos que hay varias relacionadas con los participantes de los incidentes. Todas estas siguen el mismo formato que se expuso anteriormente. Por ello, hemos decidido volver a hacer uso de la extracción de valores de estas cadenas. Posteriormente, insertaremos la moda o la media redondeada (si son valores categóricos o numéricos, respectivamente) con el mismo formato.

In [ ]:
# Calculate mode for all the participant-related columns except 'participant_age'
participant_age_group_mode = mode(extract_values(gun_violence_df['participant_age_group']))
participant_gender_mode = mode(extract_values(gun_violence_df['participant_gender']))
participant_type_mode = mode(extract_values(gun_violence_df['participant_type']))
participant_status_mode = mode(extract_values(gun_violence_df['participant_status']))
print("Moda de 'participant_age_group':", participant_age_group_mode)
print("Moda de 'participant_gender':", participant_gender_mode)
print("Moda de 'participant_type':", participant_type_mode)
print("Moda de 'participant_status':", participant_status_mode)

from statistics import mean
# Calculate mean for 'participant_age'
participant_ages = extract_values(gun_violence_df['participant_age'])
participant_ages = [int(age) for age in participant_ages]
participant_age_mean = mean(participant_ages)
participant_age_mean = round(participant_age_mean, 0)
print("Media redondeada de 'participant_age':", participant_age_mean)

In [ ]:
# Impute values for null values in participant-related columns
gun_violence_df['participant_age_group'].fillna(format_values([participant_age_group_mode]), inplace=True)
gun_violence_df['participant_gender'].fillna(format_values([participant_gender_mode]), inplace=True)
gun_violence_df['participant_type'].fillna(format_values([participant_type_mode]), inplace=True)
gun_violence_df['participant_status'].fillna(format_values([participant_status_mode]), inplace=True)
gun_violence_df['participant_age'].fillna(format_values([int(participant_age_mean)]), inplace=True)

La columna `notes` consiste de anotaciones con información adicional del incidente en formato de texto. Por ello, la mayoría de sus valores son únicos.

In [ ]:
# Show amount and proportion of unique values for 'notes' column
notes_unique_values = gun_violence_df['notes'].unique()
print("Cantidad de valores únicos en 'notes':", len(notes_unique_values))
print("Proporción de valores únicos en 'notes':", len(notes_unique_values) / gun_violence_df.shape[0])

Además, la información más relevante de estas notas (número de heridos, de muertos...) ya está incluida en otras variables del registro. Por ello, se ha decidido introducir un valor `No Notes` con el fin de no introducir ruido en la variable mediante otros métodos.

In [ ]:
gun_violence_df['notes'].fillna('No Notes', inplace=True)

Las columnas `congressional_district`, `state_house_district`, `state_senate_district` son códigos de distrito dentro de cada estado. Por tanto, aunque dos registros tengan el mismo código de distrito para el congreso (por ejemplo), si están en distintos estados tienen significados totalmente distintos.

Por esta razón, hemos decidido reemplazar los valores nulos de estas columnas por la moda de cada una de ellas dentro del estado correspondiente al registro.

In [ ]:
# Fill null values in `congressional_district`, `state_house_district`, `state_senate_district` with the mode for the corresponding state
states = gun_violence_df['state'].unique()
state_congressional_district_mode = {}
state_house_district_mode = {}
state_senate_district_mode = {}
for state in states:
    state_congressional_district_mode[state] = mode(gun_violence_df[gun_violence_df['state'] == state]['congressional_district'])
    state_house_district_mode[state] = mode(gun_violence_df[gun_violence_df['state'] == state]['state_house_district'])
    state_senate_district_mode[state] = mode(gun_violence_df[gun_violence_df['state'] == state]['state_senate_district'])

# Print modes for Alabama as an example
print("Moda de 'congressional_district' para 'Alabama':", state_congressional_district_mode['Alabama'])
print("Moda de 'state_house_district' para 'Alabama':", state_house_district_mode['Alabama'])
print("Moda de 'state_senate_district' para 'Alabama':", state_senate_district_mode['Alabama'])


In [ ]:
gun_violence_df['congressional_district'].fillna(gun_violence_df['state'].map(state_congressional_district_mode), inplace=True)
gun_violence_df['state_house_district'].fillna(gun_violence_df['state'].map(state_house_district_mode), inplace=True)
gun_violence_df['state_senate_district'].fillna(gun_violence_df['state'].map(state_senate_district_mode), inplace=True)

Aun después de esto, hay algunos estados que cuyas columnas `state_house_district` y `state_senate_district` son todas nulas, por lo que siguen teniendo valores nulos.

Para estas variables, introduciremos un valor `-1` que indicará que es desconocido.

In [ ]:
gun_violence_df['state_house_district'].fillna(-1, inplace=True)
gun_violence_df['state_senate_district'].fillna(-1, inplace=True)

Las columnas `longitude` y `latitude` son las coordenadas del accidente. Para solventar los valores faltantes, utilizaremos la media para el estado que corresponda.

In [ ]:
# Fill null values in `latitude` and `longitude` with the mean for the corresponding state
state_latitude_mean = {}
state_longitude_mean = {}
for state in states:
    state_latitude_mean[state] = gun_violence_df[gun_violence_df['state'] == state]['latitude'].mean()
    state_longitude_mean[state] = gun_violence_df[gun_violence_df['state'] == state]['longitude'].mean()

# Print means for Alabama as an example
print("Media de 'latitude' para 'Alabama':", state_latitude_mean['Alabama'])
print("Media de 'longitude' para 'Alabama':", state_longitude_mean['Alabama'])

In [ ]:
gun_violence_df['latitude'].fillna(gun_violence_df['state'].map(state_latitude_mean), inplace=True)
gun_violence_df['longitude'].fillna(gun_violence_df['state'].map(state_longitude_mean), inplace=True)

La variable `address` es una variable referente a la ubicación demasiado concreta, por lo que la gran mayoría de sus valores son únicos.

In [ ]:
address_unique_values = gun_violence_df['address'].unique()
print("Cantidad de valores únicos en 'address':", len(address_unique_values))
print("Proporción de valores únicos en 'address':", len(address_unique_values) / gun_violence_df.shape[0])

Por tanto, ya que ni la media ni la moda tienen sentido en este caso, hemos escogido utilizar un valor `Unknown` de nuevo.

In [ ]:
gun_violence_df['address'].fillna('Unknown', inplace=True)

In [ ]:
source_url_unique_values = gun_violence_df['source_url'].unique()
print("Cantidad de valores únicos en 'source_url':", len(source_url_unique_values))
print("Proporción de valores únicos en 'source_url':", len(source_url_unique_values) / gun_violence_df.shape[0])

sources_unique_values = gun_violence_df['sources'].unique()
print("Cantidad de valores únicos en 'sources':", len(sources_unique_values))
print("Proporción de valores únicos en 'sources':", len(sources_unique_values) / gun_violence_df.shape[0])

Por tanto, aplicaremos la misma técnica: la introducción de un valor `Unknown`.

Para las variables `url_source` y `sources` tenemos también una gran cantidad de valors únicos por su significado.

In [ ]:
gun_violence_df['source_url'].fillna('Unknown', inplace=True)
gun_violence_df['sources'].fillna('Unknown', inplace=True)

Por último, la variable `incident_characteristics` presenta el mismo formato previamente mencionado, común a muchas otras. Por ello, sustituiremos los valores nulos con la moda, respetando de nuevo el formato establecido.

In [ ]:
incident_characteristics_mode = mode(extract_values(gun_violence_df['incident_characteristics']))
print("Moda de 'incident_characteristics':", incident_characteristics_mode)

In [ ]:
gun_violence_df['incident_characteristics'].fillna(format_values([incident_characteristics_mode]), inplace=True)

Finalmente, conseguimos no tener valores nulos en el conjunto de datos al completo.

In [ ]:
gun_violence_df.isnull().sum()

### Tratamiento de outliers

A continuación, trataremos los valores extremos o _outliers_ en las columnas no categóricas.

In [ ]:
numerical_columns = gun_violence_df.select_dtypes(include=['int64', 'float64']).columns
numerical_columns

De las columnas de tipo numérico, `incident_id` es un identificador único. Las variables `congressional_district`, `state_house_district` y `state_senate_district`, a pesar de ser numéricas _a priori_, realmente representan una variable categórica. Por ello, no entrarán en el cálculo de valores extremos.

Además, las variables `latitude` y `longitude` son datos espaciales que, al tratar el conjunto de datos la totalidad de los Estados Unidos, nos encontramos con un contexto geográfico muy amplio. Por ello, es natural encontrar latitudes y longitudes extremas (por ejemplo, las del estado de Alaska), y modificarlas podría distorsionar la realidad.

Además, la variable `participant_age` es también una variable numérica, aunque se encuentra en el formato típico de estas variables en nuestro conjunto de datos. Por ello, extraeremos los valores individuales de cada participante.

Por tanto, se tratarán los _outliers_ de las variables `n_killed`, `n_injured`, `n_guns_involved` y `participant_age`.

In [ ]:
# Extract participant ages
participant_ages = extract_values(gun_violence_df['participant_age'])
participant_ages = [int(age) for age in participant_ages]

Primero, visualizaremos los _outliers_.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import math
 
# Identify columns to plot
numerical_columns = ['n_killed', 'n_injured', 'n_guns_involved']
numerical_data = {
    'n_killed': gun_violence_df['n_killed'],
    'n_injured': gun_violence_df['n_injured'],
    'n_guns_involved': gun_violence_df['n_guns_involved'],
    'participant_age': participant_ages
}

# Plot boxplot for numerical_data
fig, axes = plt.subplots(math.ceil(len(numerical_data) / 2), 2, figsize=(15, 15))
axes = axes.flatten()

for ax, (col, data) in zip(axes, numerical_data.items()):
    sns.boxplot(data, ax=ax)
    ax.set_title(col)

fig.tight_layout()

Los _boxplots_ nos muestran los datos que están fuera del 50% medio como puntos. Gracias a esta visualización, podemos ver que existen valores claramente erróneos, como edades de más de 100 años.

Ahora, vamos a calcular los _outliers_ como los valores que se alejan 3 o más desviaciones estándar de la media (usando el método estadístico de _z-score_).

In [ ]:
import numpy as np 

outliers = {
    'n_killed': [],
    'n_injured': [],
    'n_guns_involved': [],
    'participant_age': []
}
# Calculate outliers for numerical_data as values that are 3 standard deviations away from the mean
for col, data in numerical_data.items():
    data = np.array(data)
    mean = np.mean(data)
    std = np.std(data)
    
    threshold = 3
    col_outliers = []
    for x in data:
        z_score = (x - mean) / std
        if abs(z_score) > threshold:
            col_outliers.append(x)
    
    outliers[col] = col_outliers
    
    print("Columna: ", col)
    print("Media: ", mean)
    print("Desviación estándar: ", std)
    print("Número de outliers: ", len(col_outliers))
    print("Proporción respecto al número de filas: ", len(col_outliers) / gun_violence_df.shape[0])
    print("Outlier mínimo: ", min(col_outliers))
    print("Outlier máximo: ", max(col_outliers), "\n")

In [ ]:
# Remove outliers from gun_violence_df
for col in numerical_columns:
    gun_violence_df = gun_violence_df[~gun_violence_df[col].isin(outliers[col])]

outliers["participant_age"] = [str(age) for age in outliers["participant_age"]]
substrings_to_remove = "|".join(outliers["participant_age"])
gun_violence_df = gun_violence_df[~gun_violence_df['participant_age'].str.contains(substrings_to_remove)]

In [ ]:
participant_ages = extract_values(gun_violence_df['participant_age'])
participant_ages = [int(age) for age in participant_ages]

numerical_data = {
    'n_killed': gun_violence_df['n_killed'],
    'n_injured': gun_violence_df['n_injured'],
    'n_guns_involved': gun_violence_df['n_guns_involved'],
    'participant_age': participant_ages
}

# Plot boxplot for numerical_data without outliers
fig, axes = plt.subplots(math.ceil(len(numerical_data) / 2), 2, figsize=(15, 15))
axes = axes.flatten()

for ax, (col, data) in zip(axes, numerical_data.items()):
    data = np.array(data)
    sns.boxplot(data, ax=ax)
    ax.set_title(col)

Podemos ver que en los nuevos _boxplots_ siguen existiendo _outliers_. Esto es porque utiliza el método del IQR, mientras que nosotros hemos utilizado el método del _z-score_, por lo que son distintos. Sin embargo, esto nos muestra que hemos reducido considerablemente el número de valores extremos.

### Profiling post-limpieza

De nuevo, volvemos hacer uso del profiling para ver un resumen de nuestros datos tras la limpieza.

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(gun_violence_df, title='Gun Violence Incidents', html={'style':{'full_width':True}})
profile.to_notebook_iframe()

In [ ]:
import os
os.makedirs('../reports/descriptive_analysis', exist_ok=True)
profile.to_file(output_file='../reports/descriptive_analysis/gun_violence_incidents_postcleaning.html')

## Integración

Ahora, procederemos a resolver los problemas de representación y codificación que se presenten en el conjunto de datos, de cara a seguir un formato común con las demás fuentes.

Las variables que pueden presentarnos problemas de formato son:

- La fecha, por ser esta una variable en la que puede haber diferencias de formato dependiendo de la fuente. Además, dependiendo de la hipótesis tendremos una granularidad de día o de año.
- `gun_type`, `gun_stolen` y las variables referentes a los participantes (`participant_age`, `participant_gender`...), por tener un formato propio al conjunto de datos.

Para las segundas, hemos decidido representarlas como una lista, ya que Pandas lo permite.

In [ ]:
# Function to extract individual values from the given format, and return them as a new column with lists
def extract_values_to_lists(column):
    new_column = {column.name: []}
    for row in column:
        new_row = []
        if pd.isnull(row):
            continue
        # Remove '||' and '|' separators
        no_bars = list(filter(None, row.split('|')))
        for item in no_bars:
            if ':' in item:
                # Remove '::' and ':' separators
                no_separator = list(filter(None, item.split(':')))
                if len(no_separator) == 1:
                    print("Empty value found in column", column.name)
                    new_row.append('Unknown')
                    continue
                # participant_age column has integer values
                if column.name == 'participant_age':
                    new_row.append(int(no_separator[1]))
                else:
                    new_row.append(no_separator[1])
        new_column[column.name].append(new_row)
        
    return pd.DataFrame.from_dict(new_column)

columns_to_format = ['gun_stolen', 'gun_type', 'participant_age', 'participant_age_group', 'participant_gender', 'participant_name', 'participant_relationship', 'participant_status', 'participant_type']

for col in columns_to_format:
    gun_violence_df[col] = extract_values_to_lists(gun_violence_df[col])[col]

In [ ]:
gun_violence_df['date'].head()

La fecha sigue el formato ISO 8601 extendido, por lo que la dejaremos así. Sin embargo, dos de nuestras hipótesis tienen una granularidad de día, y las otras dos de año. Por ello, en la tarjeta de datos de las primeras se mantendrán así, mientras que en las otras dos las separaremos en año, mes y día. Esto lo haremos en la etapa de transformación, al crear las tarjetas de datos.

# Pobreza

## Análisis descriptivo

In [ ]:
# 0. load data in DataFrames
import pandas as pd
poverty_df = pd.read_csv('../data/raw/poverty_data.csv')

Pequeña muestra de los datos

In [ ]:
poverty_df.head()

Forma de los datos

In [ ]:
poverty_df.shape

Variables y sus tipos

In [ ]:
poverty_df.dtypes

Resumen de información de las variables

In [ ]:
poverty_df.info()

Descripción estadística de los datos

In [ ]:
poverty_df.describe()

Finalmente, hacemos uso de la herramienta Pandas Profiling para dar un resumen de nuestro dataset.

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(poverty_df, title='Poverty Data', html={'style':{'full_width':True}})
profile.to_notebook_iframe()

In [ ]:
import os
os.makedirs('../reports/descriptive_analysis', exist_ok=True)
profile.to_file(output_file='../reports/descriptive_analysis/poverty_incidents_precleaning.html')

## Limpieza

### Tratamiento de los valores perdidos

Primero, veremos en qué columnas existen valores nulos y cuántos de ellos hay.

In [ ]:
poverty_df.isnull().sum()

Por tanto, las columnas con valores nulos son las siguientes:

In [ ]:
null_columns = poverty_df.columns[poverty_df.isnull().any()]
null_columns

Al igual que en el anterior conjunto de datos, imputaremos la media para los valores nulos, por ser una variable numérica. Además, al ser datos demográficos que dependen del estado, utilizaremos la media del estado correspondiente al registro.

In [ ]:
# Fill missing values with the mean value of the column for the corresponding state
for col in null_columns:
    poverty_df[col] = poverty_df.groupby('state')[col].transform(lambda x: x.fillna(x.mean()))

Finalmente, conseguimos no tener valores nulos en el conjunto de datos al completo.

In [ ]:
poverty_df.isnull().sum()

### Tratamiento de outliers

A continuación, trataremos los valores extremos o _outliers_ en las columnas no categóricas.

In [ ]:
numerical_columns = poverty_df.select_dtypes(include=['int64', 'float64']).columns
numerical_columns

Descartaremos la variable `year` y trataremos los _outliers_ de las demás.

Primero, visualizaremos los _outliers_.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import math
 
numerical_columns = numerical_columns.drop('year')
# Plot boxplots for numerical columns
fig, ax = plt.subplots(math.ceil(len(numerical_columns)/3), 3, figsize=(15, 15))
for i, col in enumerate(numerical_columns):
    sns.boxplot(y=col, data=poverty_df, ax=ax[math.floor(i/3), i%3])
plt.tight_layout()

Los _boxplots_ nos muestran los datos que están fuera del 50% medio como puntos. Gracias a esta visualización, podemos ver que existen más outliers en las variables `population` y `in_poverty`. Esto es debido a que estas columnas son cantidades de habitantes, mientras que las otras son porcentajes. Esta diferencia de escalado hace que los valores extremos sean aún más extremos en estas.

Ahora, vamos a calcular los _outliers_ como los valores que se alejan 3 o más desviaciones estándar de la media (usando el método estadístico de _z-score_).

In [ ]:
import numpy as np 

# Calculate outliers for numerical_data as values that are 3 standard deviations away from the mean
outliers = {}
for col in numerical_columns:
    mean = poverty_df[col].mean()
    std = poverty_df[col].std()
    outliers[col] = poverty_df[(poverty_df[col] < mean - 3*std) | (poverty_df[col] > mean + 3*std)][col].values

    print("Columna: ", col)
    print("Media: ", mean)
    print("Desviación estándar: ", std)
    print("Número de outliers: ", len(outliers[col]))
    print("Proporción respecto al número de filas: ", len(outliers[col]) / poverty_df.shape[0])
    if len(outliers[col]) > 0:
        print("Outlier mínimo: ", min(outliers[col]))
        print("Outlier máximo: ", max(outliers[col]), "\n")
    else:
        print("\n")

Ya que los valores porcentuales no presentan _outliers_, y la renta (mediana) sólo presenta un _outlier_, no eliminaremos ningún _outlier_.

### Profiling post-limpieza

De nuevo, volvemos hacer uso del profiling para ver un resumen de nuestros datos tras la limpieza.

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(poverty_df, title='Poverty Data', html={'style':{'full_width':True}})
profile.to_notebook_iframe()

In [ ]:
import os
os.makedirs('../reports/descriptive_analysis', exist_ok=True)
profile.to_file(output_file='../reports/descriptive_analysis/poverty_incidents_postcleaning.html')